In [1]:
# import dependencies
import pandas as pd
import datetime as dt
import pymongo

In [2]:
# load the dataframe

file1=pd.read_csv('Business_Licenses.csv')
# file2=pd.read_csv('Sidewalk_Cafe_Permits.csv')

df1=pd.DataFrame(file1)
# df2=pd.DataFrame(file2)
df1.head()

FileNotFoundError: [Errno 2] File b'Business_Licenses.csv' does not exist: b'Business_Licenses.csv'

In [ ]:

df=df1[['ACCOUNT NUMBER','DOING BUSINESS AS NAME','ADDRESS','CITY','STATE','ZIP CODE','LICENSE DESCRIPTION',
       'BUSINESS ACTIVITY ID','BUSINESS ACTIVITY','APPLICATION TYPE','LICENSE TERM START DATE',
       'LICENSE TERM EXPIRATION DATE','DATE ISSUED', 'LICENSE STATUS','LATITUDE', 'LONGITUDE', 'LOCATION']]

#Select the licenses for restaurant and equivalents using the BUSINESS ACTIVITY ID
df=df.loc[(df['BUSINESS ACTIVITY ID']=='775')|
          (df['BUSINESS ACTIVITY ID']=='782') |
         (df['BUSINESS ACTIVITY ID']=='735')]

df['license_id']=df.reset_index().index

In [ ]:
df.to_csv('licenses.csv')

In [3]:
df=pd.DataFrame(pd.read_csv('licenses.csv'))

In [4]:
#Step 1 to get the list of active licences is to make a column for the year of expiration of the license
#Step 2 is to create a column named 'is_active' with Boolean Values (True if the expiration is 2019 or above, else False)
df['Expiration']=pd.to_datetime(df['LICENSE TERM EXPIRATION DATE']).dt.year
df['is_active']=[True if value>2018 else False for value in df['Expiration']]

In [5]:
# Some business will have two active licenses at the same time because they may renew a license before its term
# expires. So sort the df by the Expiration date. This will mean that those expiring the earlies will be at the bottom,
# and then you can drop the duplicated and the duplicates at the bottom will be dropped
# This ensures that you aren't dropping the most recent licenses (though its not a guarantee)

active_df=df[df['is_active']==True].sort_values('Expiration',
                                                ascending=False).drop_duplicates(subset=['ACCOUNT NUMBER'])
active_df['active_id']=active_df.reset_index().index

In [6]:

active_licenses_table=active_df[['active_id','license_id','LICENSE DESCRIPTION','BUSINESS ACTIVITY ID',
                                 'BUSINESS ACTIVITY','LICENSE TERM START DATE','LICENSE TERM EXPIRATION DATE']]

licenses_table=df[['license_id','LICENSE DESCRIPTION','BUSINESS ACTIVITY ID','BUSINESS ACTIVITY','APPLICATION TYPE',
                    'DATE ISSUED']]

active_locations_table=active_df[['active_id','DOING BUSINESS AS NAME','LATITUDE','LONGITUDE','ADDRESS','CITY','STATE',
                          'ZIP CODE']]

In [7]:
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

In [8]:
db = client.restaurantsDB


In [9]:
collection = db.active_licenses
data=active_licenses_table.to_dict(orient='records')
collection.insert_many(data)

In [10]:
collection=db.licenses
data=licenses_table.to_dict(orient='records')
collection.insert_many(data)

In [11]:
collection=db.active_locations
data=active_locations_table.to_dict(orient='records')
collection.insert_many(data)

In [12]:
help(active_locations_table.to_dict)

Help on method to_dict in module pandas.core.frame:

to_dict(orient='dict', into=<class 'dict'>) method of pandas.core.frame.DataFrame instance
    Convert the DataFrame to a dictionary.
    
    The type of the key-value pairs can be customized with the parameters
    (see below).
    
    Parameters
    ----------
    orient : str {'dict', 'list', 'series', 'split', 'records', 'index'}
        Determines the type of the values of the dictionary.
    
        - 'dict' (default) : dict like {column -> {index -> value}}
        - 'list' : dict like {column -> [values]}
        - 'series' : dict like {column -> Series(values)}
        - 'split' : dict like
          {'index' -> [index], 'columns' -> [columns], 'data' -> [values]}
        - 'records' : list like
          [{column -> value}, ... , {column -> value}]
        - 'index' : dict like {index -> {column -> value}}
    
        Abbreviations are allowed. `s` indicates `series` and `sp`
        indicates `split`.
    
    into : c

In [13]:
results = db.active_locations.find()
for result in results:
    print(result)

{'_id': ObjectId('5c6f63b232c7e6e2df25e2e4'), 'active_id': 0, '_1': "GUSI'S PIZZA", 'LATITUDE': 41.702518088000005, 'LONGITUDE': -87.55974505, 'ADDRESS': '10606 S TORRENCE AVE 1', 'CITY': 'CHICAGO', 'STATE': 'IL', '_7': 60617.0}
{'_id': ObjectId('5c6f63b232c7e6e2df25e2e5'), 'active_id': 1, '_1': 'THE BRIAR STREET THEATER', 'LATITUDE': 41.938869789, 'LONGITUDE': -87.64908649700001, 'ADDRESS': '3133 N HALSTED ST', 'CITY': 'CHICAGO', 'STATE': 'IL', '_7': 60657.0}
{'_id': ObjectId('5c6f63b232c7e6e2df25e2e6'), 'active_id': 2, '_1': 'Phils Pizza of Bridgeport inc.', 'LATITUDE': 41.83075999, 'LONGITUDE': -87.65358378399998, 'ADDRESS': '1102 W 35TH ST', 'CITY': 'CHICAGO', 'STATE': 'IL', '_7': 60609.0}
{'_id': ObjectId('5c6f63b232c7e6e2df25e2e7'), 'active_id': 3, '_1': 'SERBIAN EASTERN ORTHODOX', 'LATITUDE': 41.688094405, 'LONGITUDE': -87.532645194, 'ADDRESS': '3737 E 114TH ST 1ST', 'CITY': 'CHICAGO', 'STATE': 'IL', '_7': 60617.0}
{'_id': ObjectId('5c6f63b232c7e6e2df25e2e8'), 'active_id': 4, '_

{'_id': ObjectId('5c6f63b232c7e6e2df25ea4e'), 'active_id': 1898, '_1': "POTTER'S", 'LATITUDE': 41.880664578, 'LONGITUDE': -87.627089364, 'ADDRESS': '17 E MONROE ST 2', 'CITY': 'CHICAGO', 'STATE': 'IL', '_7': 60603.0}
{'_id': ObjectId('5c6f63b232c7e6e2df25ea4f'), 'active_id': 1899, '_1': 'HOT WOKS COOL SUSHI', 'LATITUDE': 41.881371062, 'LONGITUDE': -87.62449175200001, 'ADDRESS': '30 S MICHIGAN AVE  1', 'CITY': 'CHICAGO', 'STATE': 'IL', '_7': 60603.0}
{'_id': ObjectId('5c6f63b232c7e6e2df25ea50'), 'active_id': 1900, '_1': 'PIZANOS', 'LATITUDE': 41.881982186, 'LONGITUDE': -87.62564147, 'ADDRESS': '61-65 E MADISON ST 1ST & LL', 'CITY': 'CHICAGO', 'STATE': 'IL', '_7': 60603.0}
{'_id': ObjectId('5c6f63b232c7e6e2df25ea51'), 'active_id': 1901, '_1': 'UNIVERSITY CLUB OF CHICAGO', 'LATITUDE': 41.880924035, 'LONGITUDE': -87.62493420000001, 'ADDRESS': '76 E MONROE ST LL-12', 'CITY': 'CHICAGO', 'STATE': 'IL', '_7': 60603.0}
{'_id': ObjectId('5c6f63b232c7e6e2df25ea52'), 'active_id': 1902, '_1': 'TUXP

{'_id': ObjectId('5c6f63b232c7e6e2df25f21d'), 'active_id': 3897, '_1': 'TAQUERIA LOS GALLOS 3 INC.', 'LATITUDE': 41.793799593, 'LONGITUDE': -87.77555762700001, 'ADDRESS': '6222 S ARCHER AVE', 'CITY': 'CHICAGO', 'STATE': 'IL', '_7': 60638.0}
{'_id': ObjectId('5c6f63b232c7e6e2df25f21e'), 'active_id': 3898, '_1': "KIKI D'S B B Q HOUSE", 'LATITUDE': 41.819064669, 'LONGITUDE': -87.665073166, 'ADDRESS': '4117 S ASHLAND AVE 1ST', 'CITY': 'CHICAGO', 'STATE': 'IL', '_7': 60609.0}
{'_id': ObjectId('5c6f63b232c7e6e2df25f21f'), 'active_id': 3899, '_1': 'SUBWAY SANDWICH', 'LATITUDE': 42.008536401, 'LONGITUDE': -87.91442843899998, 'ADDRESS': '11601 W TOUHY AVE TERMINAL 3 3', 'CITY': 'CHICAGO', 'STATE': 'IL', '_7': 60666.0}
{'_id': ObjectId('5c6f63b232c7e6e2df25f220'), 'active_id': 3900, '_1': "SHIMBO'S", 'LATITUDE': 41.81410152, 'LONGITUDE': -87.670120113, 'ADDRESS': '4358 S WOOD ST', 'CITY': 'CHICAGO', 'STATE': 'IL', '_7': 60609.0}
{'_id': ObjectId('5c6f63b232c7e6e2df25f221'), 'active_id': 3901, '_

{'_id': ObjectId('5c6f63b232c7e6e2df25f9ed'), 'active_id': 5897, '_1': 'J & W FOOD & LIQUORS', 'LATITUDE': 41.677904249, 'LONGITUDE': -87.623640295, 'ADDRESS': '21 W 119TH ST 1', 'CITY': 'CHICAGO', 'STATE': 'IL', '_7': 60628.0}
{'_id': ObjectId('5c6f63b232c7e6e2df25f9ee'), 'active_id': 5898, '_1': 'PENINSULA CHICAGO, LLC', 'LATITUDE': 41.895839638000005, 'LONGITUDE': -87.624988141, 'ADDRESS': '108 E SUPERIOR ST', 'CITY': 'CHICAGO', 'STATE': 'IL', '_7': 60611.0}
{'_id': ObjectId('5c6f63b232c7e6e2df25f9ef'), 'active_id': 5899, '_1': 'MONTGOMERY CLUB', 'LATITUDE': 41.895608706, 'LONGITUDE': -87.641579165, 'ADDRESS': '500 W SUPERIOR ST', 'CITY': 'CHICAGO', 'STATE': 'IL', '_7': 60654.0}
{'_id': ObjectId('5c6f63b232c7e6e2df25f9f0'), 'active_id': 5900, '_1': 'LAS AMERICAS CORP', 'LATITUDE': 41.85711226, 'LONGITUDE': -87.65747936, 'ADDRESS': '1827 S ALLPORT ST 1ST', 'CITY': 'CHICAGO', 'STATE': 'IL', '_7': 60608.0}
{'_id': ObjectId('5c6f63b232c7e6e2df25f9f1'), 'active_id': 5901, '_1': "JOE'S PI